In [1]:
import pandas as pd
import matplotlib.pyplot as plt

pedidos = pd.read_csv('dtPedidos.csv')

pedidos['dtPedido'] = pd.to_datetime(pedidos['dtPedido'])

In [5]:
pedidos

,dtPedido,qtPedido
0,2016-09-04,1
1,2016-09-05,1
2,2016-09-13,1
3,2016-09-15,1
4,2016-10-02,1
...,...,...
98870,2018-09-29,1
98871,2018-10-01,1
98872,2018-10-03,1
98873,2018-10-16,1


In [3]:
import plotly.express as px

fig = px.histogram(pedidos, x='dtPedido')
fig.show()